In [1]:
# Code to be able to import local modules in notebooks
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
### Import dependencies
# Standard python libraries
import os
# Third party libraries
import pandas as pd
# Custom packages
from ffpackage.scraping import mfl
from ffpackage.analysis import analysis
from ffpackage.viz import viz
from appmanager.database import db

In [3]:
# Find environment variables
DATABASE_URL = os.environ.get("DATABASE_URL", None)
# sqlalchemy deprecated urls which begin with "postgres://"; now it needs to start with "postgresql://"
if DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)


In [4]:
# Retrieve basic info on all players
allPlayers = mfl.get_players()
# Retrieve Fantasy Sharks rankings
sharkRanks = mfl.get_sharkRanks()
# Retrieve Average Draft Positions
adpValues = mfl.get_adp()

# Find player ages
# Retrieve any player dobs who are already in the db
player_dobs = db.get_df('player_dobs')
# Rename the dob columns until I can get to that in the app setup
player_dobs = player_dobs.rename(columns={'PlayerID':'id_mfl', 'DOB':'dob', 'Age':'age'})
# Check for any players whose ages are not already in the db
ageNeeded = allPlayers.loc[~allPlayers['id_mfl'].isin(player_dobs['id_mfl'])]
# If there are players whose ages are not in the db, retrieve those ages from My Fantasy League
if len(ageNeeded)>0:
    # Break player list into chunks small enough for the API server
    # Specify chunk size
    n = 50
    # Slice the df where ages are needed into chunks
    list_df = [ageNeeded['id_mfl'][i:i+n] for i in range(0,ageNeeded['id_mfl'].shape[0],n)]
    # Loop over the chunks
    for i in range(len(list_df)):
        # Join the playerIDs into a string to use in building the API call
        idList = ",".join(list_df[i])
        # Retrieve the dates of birth for these players from My Fantasy League
        new_dobs = mfl.get_playerProfiles(idList)
        # Append these players to the list that already existed in the app's database
        player_dobs = player_dobs.append(new_dobs)
# Convert date columns to datetime
player_dobs['dob'] = pd.to_datetime(player_dobs['dob'])
# Convert Date of Birth to Age using datetime's relativedelta module
player_dobs['age'] = player_dobs['dob'].apply(analysis.calculate_age)

/Users/travisharry/Desktop/Coding/ff_demo2/appmanager/database/db.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)
/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_35862/3497383796.py:21: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  list_df = [ageNeeded['id_mfl'][i:i+n] for i in range(0,ageNeeded['id_mfl'].shape[0],n)]
/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_35862/3497383796.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

In [ ]:
# Merge all dfs from MyFantasyLeague API
scrape1 = scrape1.merge(player_dobs, on='PlayerID', how='left')
scrape1 = scrape1.drop(columns='DOB')
scrape1 = scrape1.merge(shark_df, on='PlayerID', how='left').merge(adp_df, on='PlayerID', how='left')
scrape1['SharkRank'].fillna(3000, inplace=True)
scrape1['ADP'].fillna(3000, inplace=True)
scrape1 = scrape1.sort_values(by=['SharkRank'])
scrape1.reset_index(inplace=True, drop=True)  

### Clean MFL data
## Select only relevant positions
scrape1 = scrape1.loc[scrape1['Position'].isin(['QB', 'WR', 'RB', 'TE', 'PK', 'Def'])]
scrape1 = scrape1.reset_index(drop=True)
## Clean Name column
to_join = scrape1['Name'].str.split(", ", n=1, expand=True)
to_join.columns = ['lname', 'fname']
to_join['Name'] = to_join['fname'] + " " + to_join['lname']
scrape1['Name'] = to_join['Name']
# Change name to Title Case
scrape1['Name'] = scrape1['Name'].str.upper()
# Drop name punctuation
scrape1['Name'] = scrape1['Name'].str.replace(".", "")
scrape1['Name'] = scrape1['Name'].str.replace(",", "")
scrape1['Name'] = scrape1['Name'].str.replace("'", "")
## Clean position column
scrape1['Position'] = scrape1['Position'].replace('Def', 'DF')
# Clean Team column
scrape1['Team'] = scrape1['Team'].replace('FA*', 'FA')
## Change column names
scrape1.columns = ['id_mfl', 'player', 'pos_mfl', 'team', 'age', 'sharkRank', 'adp']
scrape1